In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml

Make a call to the wikipedia page.  
Use BeautifulSoup to parse it.  
Get the table of values.  
Break down table into rows, then get rid of the header row.

In [2]:
site = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(site,'html.parser')
table = soup.find('table')
table_rows = table.find('tbody').findAll('tr')
table_rows = table_rows[1:len(table_rows)]

Create the dataframe with the column names.  
Iterate over the table rows, assigning the entries.  
Per the instructions, only add a row to the dataframe if a borough was assigned. The condition I used to check for this is if the anchor tag was present in the borough entry.  
A similar condition was used for determining if the neighborhood was assigned. There were some values without anchor tags so that was also taken into account in the neighborhood_assigned condition. Otherwise it would have the same value as the borough for that row in the dataframe.

In [3]:
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for tr in table_rows:
    entries = tr.findAll('td')
    postCode = entries[0].text
    borough_assigned = type(entries[1].find('a')).__name__ != 'NoneType'
    if(borough_assigned):
        borough = entries[1].find('a').text
        neighborhood_assigned = (type(entries[2].find('a')).__name__ != 'NoneType') or ("Not assigned" not in entries[2].text)
        if(neighborhood_assigned):
            if(type(entries[2].find('a')).__name__ != 'NoneType'):
                neighborhood = entries[2].find('a').text
            else:
                neighborhood = entries[2].text.strip()
        else:
            neighborhood = borough
        df_row = pd.DataFrame([postCode,borough,neighborhood]).transpose()
        df_row.columns = column_names
        df = df.append(df_row,ignore_index=True)
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M5A  Downtown Toronto   
4          M6A        North York   
5          M6A        North York   
6          M7A      Queen's Park   
7          M9A         Etobicoke   
8          M1B       Scarborough   
9          M1B       Scarborough   
10         M3B        North York   
11         M4B         East York   
12         M4B         East York   
13         M5B  Downtown Toronto   
14         M5B  Downtown Toronto   
15         M6B        North York   
16         M9B         Etobicoke   
17         M9B         Etobicoke   
18         M9B         Etobicoke   
19         M9B         Etobicoke   
20         M9B         Etobicoke   
21         M1C       Scarborough   
22         M1C       Scarborough   
23         M1C       Scarborough   
24         M3C        North York   
25         M3C        North York   
26         M4C         East York   
27         M5C  Downtown Toronto   
28         M9C         Etobicoke   
29         M9C         Etobicoke   
..         ...               ...   
179        M1W       Scarborough   
180        M1W       Scarborough   
181        M4W  Downtown Toronto   
182        M5W  Downtown Toronto   
183        M8W         Etobicoke   
184        M8W         Etobicoke   
185        M9W         Etobicoke   
186        M1X       Scarborough   
187        M4X  Downtown Toronto   
188        M4X  Downtown Toronto   
189        M5X  Downtown Toronto   
190        M5X  Downtown Toronto   
191        M8X         Etobicoke   
192        M8X         Etobicoke   
193        M8X         Etobicoke   
194        M4Y  Downtown Toronto   
195        M7Y      East Toronto   
196        M8Y         Etobicoke   
197        M8Y         Etobicoke   
198        M8Y         Etobicoke   
199        M8Y         Etobicoke   
200        M8Y         Etobicoke   
201        M8Y         Etobicoke   
202        M8Y         Etobicoke   
203        M8Y         Etobicoke   
204        M8Z         Etobicoke   
205        M8Z         Etobicoke   
206        M8Z         Etobicoke   
207        M8Z         Etobicoke   
208        M8Z         Etobicoke   

                                         Neighborhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                                        Harbourfront  
3                                         Regent Park  
4                                    Lawrence Heights  
5                                      Lawrence Manor  
6                                        Queen's Park  
7                                    Islington Avenue  
8                                               Rouge  
9                                             Malvern  
10                                    Don Mills North  
11                                   Woodbine Gardens  
12                                      Parkview Hill  
13                                            Ryerson  
14                                    Garden District  
15                                          Glencairn  
16                                         Cloverdale  
17                                          Islington  
18                                       Martin Grove  
19                                   Princess Gardens  
20                                    West Deane Park  
21                                     Highland Creek  
22                                         Rouge Hill  
23                                         Port Union  
24                                    Flemingdon Park  
25                                    Don Mills South  
26                                   Woodbine Heights  
27                                     St. James Town  
28                                  Bloordale Gardens  
29                                           Eringate  
..                                 

Group neighborhoods together if they are in the same postal code.  
First, get the postal code column and use the unique method.  
Then, make a new dataframe to hold the grouped neighborhoods.  
Next, use a for-loop to iterate over each of the postal codes.  
In a loop iteration, make a new dataframe df_post by taking the rows from df where the postal code matches the current value of code.  
Assume every neighborhood which is in the same post code is also in the same borough. Get the borough value "bor" from the first row of df_post.  
Iterate through each row of df_post to get the string value of the neighborhoods grouped together.  
Make a new dataframe row df_row with the grouped neighborhoods and append it to df_neighborhoods_grouped.

In [34]:
postCodes = df['PostalCode'].unique()
df_neighborhoods_grouped = pd.DataFrame(columns=column_names)

for code in postCodes:
    df_post = df.loc[df['PostalCode'] == code]
    bor = df_post.iloc[0][1]
    neighborhoods = ''
    for row in df_post.iterrows():
        if neighborhoods == '':
            neighborhoods=neighborhoods+row[1][2]
        else:
            neighborhoods = neighborhoods+', '+row[1][2]
    df_row = pd.DataFrame([code,bor,neighborhoods]).transpose()
    df_row.columns = column_names
    df_neighborhoods_grouped = df_neighborhoods_grouped.append(df_row, ignore_index=True)

df_neighborhoods_grouped

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A      Queen's Park   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   
12        M1C       Scarborough   
13        M3C        North York   
14        M4C         East York   
15        M5C  Downtown Toronto   
16        M9C         Etobicoke   
17        M1E       Scarborough   
18        M4E      East Toronto   
19        M5E  Downtown Toronto   
20        M1G       Scarborough   
21        M4G         East York   
22        M5G  Downtown Toronto   
23        M6G  Downtown Toronto   
24        M1H       Scarborough   
25        M2H        North York   
26        M3H        North York   
27        M4H         East York   
28        M5H  Downtown Toronto   
29        M6H      West Toronto   
..        ...               ...   
70        M2R        North York   
71        M4R   Central Toronto   
72        M5R   Central Toronto   
73        M6R      West Toronto   
74        M9R         Etobicoke   
75        M1S       Scarborough   
76        M4S   Central Toronto   
77        M5S  Downtown Toronto   
78        M6S      West Toronto   
79        M1T       Scarborough   
80        M4T   Central Toronto   
81        M5T  Downtown Toronto   
82        M1V       Scarborough   
83        M4V   Central Toronto   
84        M5V  Downtown Toronto   
85        M8V         Etobicoke   
86        M9V         Etobicoke   
87        M1W       Scarborough   
88        M4W  Downtown Toronto   
89        M5W  Downtown Toronto   
90        M8W         Etobicoke   
91        M9W         Etobicoke   
92        M1X       Scarborough   
93        M4X  Downtown Toronto   
94        M5X  Downtown Toronto   
95        M8X         Etobicoke   
96        M4Y  Downtown Toronto   
97        M7Y      East Toronto   
98        M8Y         Etobicoke   
99        M8Z         Etobicoke   

                                         Neighborhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                           Harbourfront, Regent Park  
3                    Lawrence Heights, Lawrence Manor  
4                                        Queen's Park  
5                                    Islington Avenue  
6                                      Rouge, Malvern  
7                                     Don Mills North  
8                     Woodbine Gardens, Parkview Hill  
9                            Ryerson, Garden District  
10                                          Glencairn  
11  Cloverdale, Islington, Martin Grove, Princess ...  
12             Highland Creek, Rouge Hill, Port Union  
13                   Flemingdon Park, Don Mills South  
14                                   Woodbine Heights  
15                                     St. James Town  
16  Bloordale Gardens, Eringate, Markland Wood, Ol...  
17                  Guildwood, Morningside, West Hill  
18                                        The Beaches  
19                                        Berczy Park  
20                                             Woburn  
21                                            Leaside  
22                                 Central Bay Street  
23                                           Christie  
24                                          Cedarbrae  
25                                  Hillcrest Village  
26    Bathurst Manor, Downsview North, Wilson Heights  
27                                   Thorncliffe Park  
28                           Adelaide, King, Richmond  
29                       Dovercourt Village, Dufferin  
..                                                ...  
70                                    Wi